In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anji2705","key":"cca4233ec9ee03a14901a7f4ad1862f7"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d danofer/sarcasm

Dataset URL: https://www.kaggle.com/datasets/danofer/sarcasm
License(s): copyright-authors
 81% 176M/216M [00:00<00:00, 626MB/s] 
100% 216M/216M [00:00<00:00, 684MB/s]


In [5]:
!unzip sarcasm.zip

Archive:  sarcasm.zip
  inflating: test-balanced.csv       
  inflating: test-unbalanced.csv     
  inflating: train-balanced-sarc.csv.gz  
  inflating: train-balanced-sarcasm.csv  


In [6]:
import pandas as pd

df = pd.read_csv(r"/content/train-balanced-sarcasm.csv")
df.head()


,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [7]:
df.shape

(1010826, 10)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010771 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


In [9]:
df.describe()

,label,score,ups,downs
count,1010826.0,1.010826e+06,1.010826e+06,1.010826e+06
mean,0.5,6.885676e+00,5.498885e+00,-1.458629e-01
std,0.5,4.834288e+01,4.127297e+01,3.529689e-01
min,0.0,-5.070000e+02,-5.070000e+02,-1.000000e+00
25%,0.0,1.000000e+00,0.000000e+00,0.000000e+00
50%,0.5,2.000000e+00,1.000000e+00,0.000000e+00
75%,1.0,4.000000e+00,3.000000e+00,0.000000e+00
max,1.0,9.070000e+03,5.163000e+03,0.000000e+00


In [10]:
df = df[:10000]
df = df[['label', 'comment']]
df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [11]:
df.isna().sum()

,0
label,0
comment,1


In [12]:
df.dropna(inplace = True)
df.isna().sum()

,0
label,0
comment,0


In [13]:
#removing unwanted numerals and symbols
df['comment'] = df['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [14]:
#converting the data into lowercase
def lowercase(text):
  return text.lower()

df['comment'] = df['comment'].apply(lowercase)

In [15]:
df.tail()

,label,comment
9995,1,probably a young latino boy
9996,1,dog filtergiving up
9997,1,saturday night dead amirite
9998,1,moderators not fact checkers
9999,1,she hacked the online votes


In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
def tokenize_data(text, max_length = 100):
  return tokenizer(
      text.tolist(),
      max_length = max_length,
      truncation = True,
      padding = 'max_length',
      return_tensors = 'np'
  )

In [21]:
tokenized_data = tokenize_data(df['comment'])

In [22]:
tokenized_data

{'input_ids': array([[  101, 13316,  1998, ...,     0,     0,     0],
       [  101,  2017,  2079, ...,     0,     0,     0],
       [  101,  2027,  2020, ...,     0,     0,     0],
       ...,
       [  101,  5095,  2305, ...,     0,     0,     0],
       [  101, 29420,  2015, ...,     0,     0,     0],
       [  101,  2016, 28719, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [23]:
X = tokenized_data['input_ids']
y = df['label']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [25]:
X_train.shape, X_test.shape

((7999, 100), (2000, 100))

In [26]:
class HierarchicalBERT(tf.keras.Model):
    def __init__(self, bert_model, lstm_units, cnn_filters, dense_units):
        super(HierarchicalBERT, self).__init__()
        self.bert = bert_model

        # Sentence Encoding Layer
        self.dense_sentence = tf.keras.layers.Dense(768, activation='relu')

        # Context Summarization Layer
        self.mean_pooling = tf.keras.layers.GlobalAveragePooling1D()

        # Context Encoder Layer
        self.bilstm_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, return_sequences=True))

        # CNN Layer
        self.conv = tf.keras.layers.Conv1D(cnn_filters, 2, activation='relu')
        self.pool = tf.keras.layers.GlobalMaxPooling1D()

        # Fully Connected Layer
        self.dense_fc = tf.keras.layers.Dense(dense_units, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')
    def call(self, inputs):
        # BERT Embeddings
        bert_output = self.bert(inputs)[0]  # (batch_size, seq_len, hidden_size)

        # Sentence Encoding Layer
        sentence_encoded = self.dense_sentence(bert_output)  # (batch_size, seq_len, 768)

        # Context Summarization Layer
        context_summarized = self.mean_pooling(sentence_encoded)  # (batch_size, 768)

        # Expand dimensions to match the input shape required by LSTM
        context_summarized = tf.expand_dims(context_summarized, 1)  # (batch_size, 1, 768)

        # Context Encoder Layer
        context_encoded = self.bilstm_encoder(context_summarized)  # (batch_size, 1, 2 * lstm_units)

        # Squeeze the second dimension to match the input shape required by Conv1D
        context_encoded_squeezed = tf.squeeze(context_encoded, axis=1)  # (batch_size, 2 * lstm_units)

        # Add the channels dimension to match the input shape required by Conv1D
        context_encoded_expanded = tf.expand_dims(context_encoded_squeezed, axis=-1)  # (batch_size, 2 * lstm_units, 1)

        # CNN Layer
        conv_output = self.conv(context_encoded_expanded)  # (batch_size, new_seq_len, cnn_filters)
        pooled_output = self.pool(conv_output)  # (batch_size, cnn_filters)

        # Fully Connected Layer
        dense_output = self.dense_fc(pooled_output)  # (batch_size, dense_units)

        # Output Layer
        final_output = self.output_layer(dense_output)  # (batch_size, 1)
        return final_output


In [27]:
#loading the pretrained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were 

In [28]:
#defining the hierarchical bert model
model = HierarchicalBERT(bert_model, lstm_units = 128, cnn_filters = 64, dense_units = 32)

In [29]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])

In [30]:
model.fit(X_train, y_train, epochs = 5, batch_size = 32)

Epoch 1/5


250/250 [==============================] - 216s 657ms/step - loss: 0.6624 - accuracy: 0.6266
Epoch 2/5
250/250 [==============================] - 164s 656ms/step - loss: 0.6627 - accuracy: 0.6268
Epoch 3/5
250/250 [==============================] - 162s 649ms/step - loss: 0.6619 - accuracy: 0.6268
Epoch 4/5
250/250 [==============================] - 162s 648ms/step - loss: 0.6614 - accuracy: 0.6268
Epoch 5/5
250/250 [==============================] - 162s 648ms/step - loss: 0.6617 - accuracy: 0.6268


In [31]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy : {accuracy * 100}')

63/63 [==============================] - 18s 212ms/step - loss: 0.6549 - accuracy: 0.6375
Model Accuracy : 63.749998807907104
